# OpenAI Responses API

## What is the OpenAI Responses API?

The Responses API is a new API released in March 2025. It is a combination of the traditional 
Chat Completions API and the Assistants API, providing support for:

- **Traditional Chat Completions:** Facilitates seamless conversational AI experiences.
- **Web Search:** Enables real-time information retrieval from the internet.
- **File Search:** Allows searching within files for relevant data.

Accordingly, the Assistants API will be retired in 2026. 

> **For new users, OpenAI recommends using the Responses API instead of the Chat Completions API to leverage its expanded capabilities.**

For a comprehensive comparison between the Responses API and the Chat Completions API, refer to the official OpenAI documentation: 
[Responses vs. Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

## Summary of This Notebook
This notebook provides a hands-on guide for using the **OpenAI Responses API** to analyze tweets. 
It covers essential techniques such as:

- **Creating a vector store** and uploading tweets for semantic search.
- **Using file search** to analyze private datasets.
- **Performing a web search** to retrieve the latest public information.
- **Utilizing stateful responses** to maintain conversation context.
- **Combining file and web search** to enhance retrieval-augmented generation (RAG) applications.

By the end of this notebook, users will be able to integrate OpenAI's Responses API for efficient data retrieval and analysis of structured and unstructured data.

## Install Required Libraries
To use the OpenAI Responses API, we need to install the following libraries:

- **`openai`**: Provides access to OpenAI's APIs, including the Responses API

In [1]:
pip install openai -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

In [2]:
from IPython.display import Markdown, display
import boto3
from botocore.exceptions import ClientError
import json
import io

## Retrieve Secrets from AWS Secrets Manager

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Initialize OpenAI Client

In [4]:
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']

client = OpenAI(api_key=openai_api_key)

## File Search API

### Introduction to File Search
File search API enables efficient retrieval of relevant information 
from uploaded files by leveraging vector-based indexing. This feature is particularly useful 
for searching large datasets, extracting insights, and improving retrieval-augmented generation (RAG) applications.

Unlike traditional keyword-based searches, the Responses API uses embeddings 
to identify semantically relevant content, making it ideal for analyzing structured 
and unstructured text data (OpenAI, 2025).

For more details, visit the official OpenAI documentation: 
[File Search in Responses API](https://platform.openai.com/docs/guides/tools-file-search).

### Create a Vector Store

In [5]:
vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

vs_69126391be24819195c1358cb01849c8


### Upload Files

In [6]:
with open('tweet_text.json', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-CHy2a2pLmPMpRCjGfKuayC


### Attach File to Vector Store

In [7]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-CHy2a2pLmPMpRCjGfKuayC


### Query the Vector Store

In [8]:
query = "the latest development in generativeAI"

In [9]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

## OpenAI Response API

### Simple Response

In [10]:
simple_response = client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": query
      }
  ]
)

In [11]:
display(Markdown(simple_response.output_text))

As of the latest updates in 2023, generative AI has been evolving rapidly with several key developments:

1. **Improved Models**: Models like GPT-4 and others have become more sophisticated, offering enhanced performance in language understanding and generation across various tasks.

2. **Multimodal Capabilities**: There's a significant push towards multimodal models capable of processing and generating text, images, and other types of data, allowing for more comprehensive applications.

3. **Personalization**: Efforts to create more personalized AI experiences are gaining traction, with models being fine-tuned for specific user preferences and contexts.

4. **Ethical and Safe AI**: Ongoing work is being done to enhance the ethical use of AI, focusing on reducing bias, improving transparency, and ensuring safe interactions.

5. **Creative Applications**: Generative AI is increasingly being used in creative fields, including art, music, and content creation, often collaborating with human creators.

6. **Real-time Applications**: AI is now being employed for real-time applications, such as dynamic content generation in video games and interactive storytelling.

7. **Integration in Business**: Businesses are leveraging generative AI for customer service, content marketing, and automated data analysis to improve efficiency and customer engagement.

These advancements are significantly shaping the landscape of AI and expanding its potential uses across diverse sectors.

### File Search Response

In [12]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [13]:
display(Markdown(file_search_response.output_text))


The latest developments in generative AI include several key trends and innovations:

1. **Agentic Workflows**: Amazon Web Services is focusing on agentic workflows, which are likely to enhance the automation and efficiency of AI systems.

2. **AI in Supply Chain**: Atos has developed an AI-powered Supply Chain Disruption Analysis tool using generative AI, SAP BTP, and AWS Bedrock to assess risks and boost resilience.

3. **Market Growth**: The global generative AI market is projected to reach $1.18 billion this year, highlighting its rapid growth and adoption across industries.

4. **Creative Industries**: Generative AI is transforming creative industries by enabling the creation of content, designs, and strategies, thus reshaping business innovation.

5. **Enterprise Applications**: IBM's Watsonx is bringing generative AI to enterprises with tools for secure and responsible deployment, allowing for custom large language models to enhance customer engagement and streamline processes.

These developments indicate a broadening application of generative AI across various sectors, from supply chain management to creative industries and enterprise solutions.

## Web Search API

### Introduction to Web Search
The OpenAI Web Search tool allows models to retrieve real-time information from the internet. 
This capability is particularly useful for obtaining up-to-date data, fact-checking, and expanding knowledge 
without relying solely on pre-trained information. 

By leveraging OpenAI's web search functionality, the Responses API can fetch external data 
and provide accurate, relevant results in real time (OpenAI, 2025). 
This feature enhances applications that require the latest insights, such as news aggregation, research, 
or dynamic content generation.

For more details, visit the official OpenAI documentation: 
[Web Search in Responses API](https://platform.openai.com/docs/guides/tools-web-search).

### Perform Web Search

In [14]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [15]:
display(Markdown(web_search_response.output_text))

Below is a structured and thorough overview (approx. 770 words) of the latest developments in generative AI as of **Monday, November 10, 2025**. Each paragraph includes at least two citations, and the response is formatted in Markdown (per development instructions).

---

##  Major Model Announcements & Breakthroughs

- **OpenAI’s Sora 2 (Text-to-Video + Social Feed)**  
  On **September 30, 2025**, OpenAI introduced **Sora 2**, a next-generation text-to-video model that enables synchronized dialogue, improved scene consistency across longer clips, and narrative accuracy from complex prompts. They published the **Sora 2 System Card**, detailing updated safety protocols, training data transparency, and post-training limitations—a move highlighting increased accountability in frontier AI deployment ([champaignmagazine.com](https://champaignmagazine.com/2025/10/05/ai-by-ai-weekly-top-5-september-29-october-5-2025/?utm_source=openai)).

- **Sora Android Launch**  
  In early November (Nov 6), the **Sora Android app** launched with explosive adoption— around **470,000 downloads** on day one, surpassing its iOS debut (~360,000 downloads) by 327%. This reflects surging consumer interest in AI-generated video content ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).

- **OpenAI’s Aardvark: AI Security Researcher**  
  On **October 30, 2025**, OpenAI unveiled **Aardvark**, an autonomous security researcher powered by GPT‑5. Currently in private beta, Aardvark achieves ~92% recall on seeded test repositories and has already generated 10 CVE-numbered security vulnerability disclosures — a significant step toward AI-driven cybersecurity workflows ([aiwebbiz.com](https://aiwebbiz.com/blog/top-5-ai-news-of-the-week-november-2025/?utm_source=openai)).

- **Emerging “Thinking” Model: GPT‑5.1 “Thinking”**  
  Recent leaks suggest OpenAI is preparing a new model variant, **GPT‑5.1 “Thinking”**, designed for enhanced reasoning and strategic responses. It could compete with Google’s Gemini 3 Pro (a model expected to support a **1-million-token context window**). GPT‑5.1 may first roll out to premium users via ChatGPT’s backend ([tomsguide.com](https://www.tomsguide.com/ai/chatgpt/openais-leaked-gpt-5-1-thinking-model-could-outsmart-gemini-3-pro-heres-why-that-matters?utm_source=openai)).

---

##  Competing Models & Ecosystem Advances

- **Google’s Gemini 3 Pro Preview & Deep Research Integration**  
  A preview of **Gemini 3 Pro** appeared on Google’s Vertex AI platform, indicating an imminent release. The model is expected to support up to a **1-million-token context window**, enabling more complex and prolonged interactions ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai)).

  Concurrently, Google launched a "**Deep Research**" feature within Gemini AI. This allows the model to leverage data from user Gmail, Drive, Chat, and Search to build detailed, personalized research outputs ([medium.com](https://medium.com/%40CherryZhouTech/ai-news-november-1-7-2025-10-ai-advances-you-cant-miss-this-week-e2e86d331bd0?utm_source=openai)).

- **Google’s Nano Banana 2 (AI Image Upgrade)**  
  Google is readying **Nano Banana 2**, an upgraded version of its viral image generation tool initially known as Nano Banana (Gemini 2.5 Flash Image). The new model enhances speed, style diversity, and includes watermarking for compliance. It will integrate closely with the Gemini 3 series ([tomsguide.com](https://www.tomsguide.com/ai/ai-image-video/next-gen-nano-banana-2-could-bring-4k-ai-image-generation-to-your-phone-heres-whats-coming?utm_source=openai)).

- **Microsoft’s MAI‑Image‑1 Launch**  
  Just **5 days ago (Nov. 5, 2025)**, Microsoft released **MAI‑Image‑1**, its first in-house text-to-image generator. Available via Bing Image Creator and Copilot, the model offers photorealistic and detailed image outputs. Despite still trailing OpenAI’s offerings by a few months, it delivers faster iterations and more creative control. Microsoft still maintains integration rights with OpenAI tools like DALL·E 3 and GPT‑4o through 2032 ([windowscentral.com](https://www.windowscentral.com/artificial-intelligence/microsoft-copilot/microsoft-launches-mai-image-1?utm_source=openai)).

---

##  Generative AI in Biotech & Video

- **AI-Designed Antibodies with RFantibody**  
  A breakthrough study published in *Nature* by Nobel laureate David Baker and his team introduced **RFantibody**, a generative AI system capable of designing functional antibodies from scratch. The model accelerates early-stage drug discovery—particularly for complex diseases such as cancer—by producing antibodies that bind to specific cancer-related proteins without requiring animal testing. While later validation stages still require clinical trials and regulatory review, the development represents a significant step in ethical and efficient biopharma innovation ([ft.com](https://www.ft.com/content/328a3211-6f2f-471e-b7bd-eb3c1a768f1c?utm_source=openai)).

- **Runway Gen‑4 (Text-to-Video AI)**  
  Released in March 2025, **Runway Gen‑4** is a text-to-video model capable of generating 5–10 second clips at 720p resolution with consistent characters and simulated camera movements like pans and zooms. It employs a diffusion-based transformer architecture and enables reference image–based scene creation. While character consistency holds within individual clips, it still falters across independently generated sequences ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gen-4_%28AI_image_and_video_model%29?utm_source=openai)).

---

##  Market Trends & Enterprise Adoption

- **Market Growth & Infrastructure Spending**  
  The generative AI market has surged across hardware, foundation models, and development platforms, enjoying triple-digit growth. Expected AI-related spending in 2025 is projected to reach **US$400 billion**, with hardware revenues (mostly GPU-based systems) hitting **US$132.3 billion** in 2024. Foundation models contributed ~US$4.1 billion (excluding consumer apps), while development platforms added ~US$17 billion ([globenewswire.com](https://www.globenewswire.com/news-release/2025/08/19/3135499/0/en/Generative-AI-Market-Report-2025-US-400-Billion-in-AI-Related-Spending-Anticipated-by-2025.html?utm_source=openai)).

- **Enterprise Gen AI as Core Infrastructure**  
  Generative AI is increasingly treated as core infrastructure rather than experimental tools. Across knowledge work, developers report 30–50% reductions in coding time using tools like GitHub Copilot; customer support systems autonomously resolve over 60% of Tier 1 queries. Even regulated sectors (banking, pharmaceuticals) adopt domain-fine-tuned GenAI for functions like compliance, research, and underwriting ([globenewswire.com](https://www.globenewswire.com/news-release/2025/08/27/3140198/28124/en/Generative-AI-Company-Benchmark-Analysis-Report-2025-NVIDIA-Microsoft-and-Google-Dominate-with-Advanced-GPUs-Cloud-Integration-and-Cutting-Edge-AI-Models.html?utm_source=openai)).

---

##  Summary Table of Key Recent Developments

| Area                   | Highlight                                                                 |
|------------------------|---------------------------------------------------------------------------|
| Text-to-video          | OpenAI’s **Sora 2** launch and app success; Runway **Gen‑4** model         |
| AI Agents              | OpenAI’s **Aardvark** for security; Google’s **Deep Research** feature     |
| Image Generation       | Microsoft’s **MAI‑Image‑1**; Google’s **Nano Banana 2**                    |
| Multimodal LLMs        | Leaks on **GPT‑5.1 “Thinking”** vs. Google’s **Gemini 3 Pro**              |
| Biotech Applications   | **RFantibody** for AI-designed antibodies                                 |
| Market Momentum        | Massive infrastructure investment and enterprise adoption growth           |

---

### Final Thoughts

As of **November 10, 2025**, generative AI is at a dynamic inflection point. Frontline research and product releases now tackle complex tasks—from autonomous video creation to real-world security, to antibody design—while infrastructure and enterprise usage scale rapidly. This dual trajectory—technical innovation and foundational adoption—signals that gen AI is no longer niche but essential across domains.

If you'd like, I can dive deeper into any of these developments—just let me know which one interests you most.

### Stateful Response

The OpenAI Responses API includes a stateful feature that enables continuity in interactions. 
By using the `response_id`, a conversation can persist across multiple queries, 
allowing users to refine or expand upon previous searches. This is particularly useful for iterative research, 
dynamic content generation, and applications that require follow-up queries based on prior responses.

In [16]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

Below is a structured and thorough overview (approx. 770 words) of the latest developments in genera

### Continue Query with Web Search

In [17]:
continue_query = 'find different news'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [18]:
display(Markdown(continue_search_response.output_text))

Here are several *other* **recent and diverse developments** in generative AI, reflecting the latest advancements across industries and applications—as of **November 10, 2025**. Each paragraph highlights different areas and includes at least two citations.

---

**TIME launches a generative AI-powered journalistic agent**  
On **November 10, 2025**, *TIME* unveiled the **TIME AI Agent**, a sophisticated generative AI platform built in partnership with Scale AI. This tool transforms the way audiences interact with news by enabling personalized summaries, audio reports, translations, and interactive exploration of content—all grounded in real journalism and equipped with robust moderation to ensure output integrity and editorial standards.([time.com](https://time.com/7332572/the-story-behind-the-time-ai-agent/?utm_source=openai))

---

**Adobe expands generative creativity with Firefly upgrades**  
At Adobe MAX 2025, Adobe introduced powerful enhancements to its Firefly AI creative suite. New features include the ability to train Firefly on custom models using only six images, enabling stylistic or character-specific generation; “Generate Soundtrack” and “Generate Speech” capabilities for auto-generating music and expressive voiceovers in videos; and a browser-based multi-track Firefly video editor. Adobe also launched Project Moonlight—a system to carry user context across apps—and previewed integration between Firefly and ChatGPT.([wired.com](https://www.wired.com/story/adobe-max-2025-firefly-photoshop-updates?utm_source=openai))

---

**Generative AI’s explosive growth in the gaming industry**  
Recent analysis reveals that fully **one in five games released on Steam in 2025** employ generative AI elements—a staggering **681% increase year-over-year**. Among the platform’s roughly 114,000 titles, 7,818 disclose GenAI usage, primarily for visual assets (60%), but also for audio, narrative, marketing, and real-time gameplay enhancements.([tomshardware.com](https://www.tomshardware.com/video-games/pc-gaming/1-in-5-steam-games-released-in-2025-use-generative-ai-up-nearly-700-percent-year-on-year-7-818-titles-disclose-genai-asset-usage-7-percent-of-entire-steam-library?utm_source=openai))

---

**Meta’s major investment in Scale AI signals ambition in generative AI**  
In June 2025, **Meta invested $14.8 billion to acquire a 49% stake in Scale AI**. This strategic move brings Scale’s CEO, Alexandr Wang, into a leadership role at Meta to push forward superintelligent AI development. The acquisition strengthens Meta’s access to high-quality labeled data and underscores its renewed focus on catching up to competitors like OpenAI and Google in generative AI.([washingtonpost.com](https://www.washingtonpost.com/technology/2025/06/10/ai-meta-scale-google-openai/?utm_source=openai))

---

**SP Global warns: rapid generative AI growth brings mixed outcomes**  
A **SP Global report** from late October 2025 notes that while adoption of generative AI has accelerated dramatically—with most organizations now using it—many projects are failing. Abandonment rates for proof-of-concept initiatives climbed from 17% to 42%, with only 19% reporting strong positive impact. Common challenges include cost, accuracy, privacy, and trust concerns.([spglobal.com](https://www.spglobal.com/market-intelligence/en/news-insights/research/2025/10/generative-ai-shows-rapid-growth-but-yields-mixed-results?utm_source=openai))

---

**Music industry adapts: Universal Music Group partners with Stability AI**  
On **October 30, 2025**, **Universal Music Group (UMG)** and **Stability AI** announced a landmark collaboration to develop AI music tools using **licensed, ethical training data** from UMG’s catalog. The aim is to create AI tools that respect artist rights while enabling new creative possibilities—marking a shift from opposition to proactive design in AI music generation.([aiwebbiz.com](https://aiwebbiz.com/blog/top-5-ai-news-of-the-week-november-2025/?utm_source=openai))

---

**Google continues advancing Gemini ecosystem with Deep Research and Nano Banana 2**  
In early November 2025, **Google added “Deep Research” to Gemini AI**—allowing it to draw on Gmail, Drive, Chat, and Search to deliver richer, personalized reports. Simultaneously, the next-generation **Nano Banana 2** image model is being prepared, promising faster generation, enhanced style diversity, and built-in watermarking, integrated closely with the upcoming Gemini 3.0 series.([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

---

**India launches BharatGen multimodal LLM and OpenAI Academy**  
In early June 2025, India unveiled **BharatGen**, a multilingual, multimodal LLM supporting 22 regional languages, designed for healthcare, education, agriculture, and governance. Alongside that, **OpenAI and IndiaAI Mission** launched the first overseas **OpenAI Academy**—offering training in regional languages, API credits, hackathons, and GPU access, aiming to upskill 1 million teachers and support a broad developer community.([linkedin.com](https://www.linkedin.com/pulse/top-5-generative-ai-news-updates-from-week-23-2025-1st-shankar-1tj5c?utm_source=openai))

---

**Summary of developments by domain**  
• Journalism: TIME AI Agent brings interactive generative capabilities to news consumption.  
• Creativity: Adobe Firefly expands into audio, speech, multi-track editing, and contextual workflow.  
• Gaming: GenAI use in games surges, especially for visuals and gameplay.  
• Industry consolidation: Meta’s stake in Scale AI accelerates its generative AI ambitions.  
• Enterprise: SP Global cautions that AI rollout often falters due to execution challenges.  
• Music: Ethical AI music co-development signals industry adaptation.  
• Platforms: Google’s Gemini adds deeper research and image generation improvements.  
• Education & policy: India’s BharatGen and OpenAI Academy show national scale AI deployment.

---

If there's a specific area—such as music, gaming, enterprise adoption, or generative AI in journalism—you'd like to explore next, I'd be happy to dive deeper into that.

### Combining File Search and Web Search

This is an example of using file search to analyze private data and web search to retrieve public or the latest data. 
The Responses API allows developers to integrate these tools to enhance retrieval-augmented generation (RAG) applications. 
By combining file search with web search, users can leverage structured internal knowledge while also retrieving real-time 
information from external sources, ensuring comprehensive and up-to-date responses. 

In [19]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [20]:
display(Markdown(combined_search_response.output_text))

Recent developments in generative AI include several key advancements and applications:

1. **Enterprise Integration**: IBM's Watsonx is bringing generative AI to enterprises with tools designed for secure and responsible deployment. This allows teams to build custom large language models to enhance customer engagement and streamline processes.

2. **Market Growth**: The global generative AI market is expected to reach $1.18 billion this year, highlighting the rapid adoption and investment in this technology.

3. **Creative Industries**: Generative AI is transforming creative industries by enabling the creation of content, designs, and strategies, reshaping how businesses innovate.

4. **Healthcare**: In healthcare, generative AI is being used to support doctors with data interpretation and preliminary analysis, balancing AI assistance with human oversight to ensure safety and efficacy.

5. **Finance**: Generative AI is being embedded in banking systems to improve services like chatbots, fraud detection, and robo-advisors.

These developments illustrate the broad impact of generative AI across various sectors, driving innovation and efficiency.

# 🧩 Try It Yourself: Two-Step RAG (Private Data + Combined Search)

## Step 1 — Upload & Create Vector Store
1. Upload a short text file (e.g., `my_notes.txt`) to your notebook instance.  
2. Create a **vector store** and **ingest** your uploaded file.  
3. Run a simple test query to verify retrieval:  

In [42]:
with open('IA343_1.txt', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-Nwmkm2UcY7RHMuB81ALbc7


In [43]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-Nwmkm2UcY7RHMuB81ALbc7


In [44]:
query2 = "the latest information about the government shutdown"

In [45]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query2
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

SKY NEWS LIVE
5:14PM Monday, November 10th, 2025
SubscribeLog in
Sky News Australia
Australia
W
 Relevant score: 0.9300348206292677
SKY NEWS LIVE
5:14PM Monday, November 10th, 2025
SubscribeLog in
Sky News Australia
Australia
W
 Relevant score: 0.9300348206292677
SKY NEWS LIVE
5:14PM Monday, November 10th, 2025
SubscribeLog in
Sky News Australia
Australia
W
 Relevant score: 0.9300348206292677
SKY NEWS LIVE
5:14PM Monday, November 10th, 2025
SubscribeLog in
Sky News Australia
Australia
W
 Relevant score: 0.9300348206292677
Under a deal struck with a group of Democrats, Republicans would agree to a vote in December on exte
 Relevant score: 0.7806489306716703


## Step 2 — Combine File Search with Web Search
1. Enable both **file_search** and **web_search** in the Responses API.  
2. Use a prompt that asks the model to merge insights from both sources.  
   > Example: “Using my uploaded notes and the latest web information, summarize the current trends on this topic.”  
3. Review how the answer from your file and **current info** from the web.

✅ You’ve created a RAG system that combines **private** and **public** data for comprehensive, up-to-date analysis.


In [ ]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query2,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [ ]:
display(Markdown(web_search_response.output_text))